# Configuration Initial et setup

In [5]:
!pip install geopandas
!pip install delta


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
  Created wheel for delta: filename=delta-0.4.2-py3-none-any.whl size=2914 sha256=6144f473517bf4801888882eb07b3b7ea6d540138db6d02fb74b97eb9ba24421
  Stored in directory: /root/.cache/pip/wheels/a8/86/24/a486f14769cf86a2a9ce6b589a82b7414b14657c6fd515dc75
Successfully built delta

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [6]:
import sys
import os
import zipfile
import pandas as pd
import geopandas as gpd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta import *
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [8]:
builder = SparkSession.builder \
    .appName("01_ingestion_bronze") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.parquet.compression.codec", "snappy")

In [11]:
# Configuration des chemins
base_path = "/dbfs/mnt/lakehouse"
bronze_path = f"{base_path}/bronze"

# Création des répertoires Delta
directories = [
    f"{bronze_path}/weather",
    f"{bronze_path}/taxi_trips", 
    f"{bronze_path}/taxi_zones",
    f"{bronze_path}/taxi_zones_shp"
]


In [12]:
for dir_path in directories:
    os.makedirs(dir_path, exist_ok=True)

print("✅ Configuration initiale terminée")
print(f"📁 Répertoire base: {base_path}")
print(f"📁 Répertoire bronze: {bronze_path}")

✅ Configuration initiale terminée
📁 Répertoire base: /dbfs/mnt/lakehouse
📁 Répertoire bronze: /dbfs/mnt/lakehouse/bronze


# Fonctions Utilitaires

In [13]:
def log_ingestion(source_name, file_count, record_count, success=True, error_msg=None):
    """Log des opérations d'ingestion"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    log_data = {
        "timestamp": timestamp,
        "source": source_name,
        "file_count": file_count,
        "record_count": record_count,
        "success": success,
        "error_message": error_msg if error_msg else ""
    }
    
    # Sauvegarde dans un DataFrame Spark
    log_df = spark.createDataFrame([log_data])
    
    # Écriture ou ajout au log Delta
    log_path = f"{bronze_path}/ingestion_log"
    
    if os.path.exists(f"{log_path}/_delta_log"):
        log_df.write.format("delta").mode("append").save(log_path)
    else:
        log_df.write.format("delta").save(log_path)
    
    if success:
        print(f"✅ {source_name}: {record_count:,} enregistrements ingérés")
    else:
        print(f"❌ {source_name}: Échec - {error_msg}")

def extract_zip(zip_path, extract_to):
    """Extrait un fichier ZIP"""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"📦 Fichier ZIP extrait: {zip_path} -> {extract_to}")

def validate_schema(df, expected_columns):
    """Valide le schéma d'un DataFrame"""
    actual_columns = set(df.columns)
    expected_set = set(expected_columns)
    
    missing = expected_set - actual_columns
    extra = actual_columns - expected_set
    
    if missing:
        print(f"⚠️  Colonnes manquantes: {missing}")
    if extra:
        print(f"⚠️  Colonnes supplémentaires: {extra}")
    
    return len(missing) == 0

# Ingestion des données meteos(csv)

In [14]:
print("="*80)
print("🌤️  INGESTION DES DONNÉES MÉTÉO")
print("="*80)

# Chemin du fichier CSV
weather_csv_path = "/dbfs/FileStore/shared_uploads/votre_email@domaine.com/central_park_weather_2024.csv"
weather_bronze_path = f"{bronze_path}/weather"

try:
    # Lecture avec Pandas pour inspection
    weather_pd = pd.read_csv(weather_csv_path)
    print(f"📊 Lecture du CSV: {len(weather_pd):,} lignes, {len(weather_pd.columns)} colonnes")
    print("📋 Aperçu des colonnes:", weather_pd.columns.tolist())
    print("📅 Plage temporelle:", weather_pd['time'].min(), "à", weather_pd['time'].max())
    
    # Ajout de métadonnées
    weather_pd['ingestion_date'] = datetime.now().date()
    weather_pd['source_file'] = 'central_park_weather_2024.csv'
    weather_pd['data_type'] = 'weather_observation'
    
    # Conversion en Spark DataFrame
    weather_df = spark.createDataFrame(weather_pd)
    
    # Correction des types de données
    weather_df = weather_df \
        .withColumn("time", to_timestamp(col("time"))) \
        .withColumn("temp", col("temp").cast("float")) \
        .withColumn("rhum", col("rhum").cast("int")) \
        .withColumn("prcp", col("prcp").cast("float")) \
        .withColumn("wspd", col("wspd").cast("float")) \
        .withColumn("pres", col("pres").cast("float"))
    
    # Schéma attendu
    expected_weather_cols = ['time', 'temp', 'rhum', 'prcp', 'wspd', 'pres', 
                            'ingestion_date', 'source_file', 'data_type']
    
    if validate_schema(weather_df, expected_weather_cols):
        # Écriture en Delta avec partition par mois
        weather_df = weather_df \
            .withColumn("year", year(col("time"))) \
            .withColumn("month", month(col("time"))) \
            .withColumn("day", dayofmonth(col("time")))
        
        print("💾 Écriture en Delta...")
        weather_df.write \
            .format("delta") \
            .mode("overwrite") \
            .partitionBy("year", "month") \
            .save(weather_bronze_path)
        
        # Vérification
        delta_weather = spark.read.format("delta").load(weather_bronze_path)
        record_count = delta_weather.count()
        
        log_ingestion(
            source_name="weather_data",
            file_count=1,
            record_count=record_count,
            success=True
        )
        
        print("📊 Statistiques du Delta:")
        delta_weather.select("year", "month").distinct().orderBy("year", "month").show()
        
except Exception as e:
    error_msg = f"Erreur lors de l'ingestion météo: {str(e)}"
    log_ingestion(
        source_name="weather_data",
        file_count=1,
        record_count=0,
        success=False,
        error_msg=error_msg
    )
    raise

🌤️  INGESTION DES DONNÉES MÉTÉO


Py4JJavaError: An error occurred while calling o68.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:873)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:260)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:243)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.inter